# Semestrální práce - Radomír Čech

Analýza dat sčítání lidu 2021

### Dostupnost dat

- jedná se o otevřená data (vysvětlení otevřených dat bude v textu)
- primárně jsou data rozdělená podle typů územních úrovní, některé více do podrobna, některé méně, kategorie (data) jsou ale stejné
- data jsou strukturovaná do CSV souborů s techinickými schématy v podobě JSON souborů, které obsahují vysvětlení atributů a jejich datové typy, tyto soubory jsou dostupné online na daném URL
- CSV formát je určité jednodušší pro práci s daty, ale vyžaduje stažení všech dat (oproti API, kde je moźné si stáhnout určitou část dat)
- není dostupný žádný náhled dat pro představu
- data se dají jednoduše vyhledat na internetu, stačilo googlit "vysledky scitani 2021"

### Struktura dat

- jedná se o CSV soubory - textový soubor s hodnotami oddělenými oddělovačem, v tomto případě čárkou
- soubory jsou různé velikosti, občas desítky KB, občas desítky MB, někde i přes 100 MB
- data lze otevřít v jakémkoli textovém editoru, ale je vhodné je převést do prezentovatelné podoby (např. tabulky)
- nejjednodušším nástrojem je pravděpodobně MS Excel
    - dokáže zobrazit náhled CSV souboru v původní textové podobě
    - pro převod do tabulkové podoby je nutné otevřít prázdný sešit a v kartě "Data" použít "Z textu/CSV"
    - někdy nutné nastavit kódování na UTF-8 a správný oddělovač (čárka)
    - v závislosti na velikosti datasetu se mohou data převádět i několik minut
    - výsledkem je kontigenční tabulka, která podporuje filtrování a jiné funkce
- pro programátorské řešení jsem využil databázový systém
    - pokud je třeba v datech vyhledávat programově, není CSV vhodný a efektivní formát, protože by vyžadoval sekvenční průchod všech dat
    - bylo rozhodnuto o využití NoSQL databázového systému MongoDB, NoSQL je vhodné díky rychlosti práce s daty + data neobsahují vzájemné relace
    - porovnání dat v CSV formátu a NoSQL formátu
        - příklad: obyvatelstvo podle pohlaví a pětiletých věkových skupin
            - velikost původních CSV dat: 65,5 MB
            - velikost kontigenční tabulky v MS Excel: 29 MB (o 56 % menší)
            - velikost MongoDB kolekce + index: 20,7 + 4,6 = 25,3 MB (o 62 % menší)

### Analýza a práce s daty

- pro jednodušší operace je vhodné používat kontigenční tabulku v MS Excel
    - např. zjištění nejlidnatějšího kraje

- pro náročnější operace je vhodné používat programátorské řešení přes MongoDB
    - např. ...
    - pro procházení dat v databázi se doporučuje program přímo od tvůrců MongoDB - MongoDB Compass, který je dostupný na Windows, MacOS a Linux
- atributy v každém datasetu se mohou mírně lišit, zde je příklad atributů v datasetu "Obyvatelstvo podle věku a pohlaví":

    | SLOUPEC     | VÝZNAM                                                                                                                   |
    |-------------|--------------------------------------------------------------------------------------------------------------------------|
    | idhod       | unikátní   identifikátor údaje Veřejné databáze ČSÚ                                                                      |
    | hodnota     | zjištěná   hodnota                                                                                                       |
    | ukaz_kod    | kód   ukazatele (statistické proměnné)                                                                                   |
    | vek_cis     | číselník   pro věkovou skupinu; pokud není uveden, jedná se o celkový údaj bez rozlišení   věkové skupiny                |
    | vek_kod     | kód   položky z číselníku pro věkovou skupinu; pokud není uveden, jedná se o   celkový údaj bez rozlišení věkové skupiny |
    | pohlavi_cis | číselník   pro pohlaví; pokud není uveden, jedná se o celkový údaj bez rozlišení pohlaví                                 |
    | pohlavi_kod | kód   položky z číselníku pro pohlaví; pokud není uveden, jedná se o celkový údaj   bez rozlišení pohlaví                |
    | uzemi_cis   | kód   číselníku pro území                                                                                                |
    | uzemi_kod   | kód   položky číselníku pro území                                                                                        |
    | sldb_rok    | rok   sčítání ve formátu YYYY                                                                                            |
    | sldb_datum  | rozhodné   datum sčítání ve formátu YYYY-MM-DD                                                                           |
    | ukaz_txt    | text   ukazatele                                                                                                         |
    | vek_txt     | text   položky z číselníku pro věkovou skupinu                                                                           |
    | pohlavi_txt | text   položky z číselníku pro pohlaví                                                                                   |
    | uzemi_txt   | text   položky z číselníku pro území                                                                                     |

<br>

- problematické mohou být atributy jako `vek_kod` nebo `uzemi_cis`, tyto atributy obsahují CIS hodnoty reprezentující hodnotu z daného číselníku
    - např. `uzemi_cis` popisuje územní úroveň dané hodnoty (třeba okres, kraj...), takže pokud chceme pracovat třeba s daty okresů, musíme znát tyto hodnoty a podle nich filtrovat
    - některé z těchto atributů mají v datech i svůj textový popis, např. `uzemi_kod`, který má hodnotu '500011', má svůj slovný popis v atributu `uzemi_txt` s hodnotou 'Želechovice nad Dřevnicí'
    - některé atributy nemají žádný textový popis, např. `uzemi_cis`
    - příklad hodnot pro `uzemi_cis` a jejich vysvětlení:

        | KÓD | NÁZEV                                       |
        |-----|---------------------------------------------|
        | 97  | Stát                                        |
        | 99  | Region soudržnosti                          |
        | 100 | Kraj                                        |
        | 101 | Okres                                       |
        | 65  | Správní obvody obcí s rozšířenou působností |
        | 72  | Správní obvod v hlavním městě Praze         |
        | 43  | Obec a vojenský újezd                       |
        | 44  | Městský obvod, městská část                 |

<br>

- každý záznam obsahuje také své vlastní unikátní ID, to ale není nijak využitelné v našem případě

- některé záznamy obsahují prázdné hodnoty, to má svůj význam, např. když atribut `pohlavi_kod` má prázndnou hodnotu, znamená to, že tento záznam uchovává hodnotu pro všechna pohlaví (hodnota za muže a ženy dohromady), to samé platí i např. pro `vek_kod`
- některé atributy jsou trochu matoucí, atributy `ukaz_kod`, `vek_cis`, `pohlavi_cis` jsou vždy stejné, tudíž nepředstavují žádný význam pro tento dataset ("Obyvatelstvo podle věku a pohlaví"), navíc nejsou čitelné, protože se jedná o kódy, to se ale může lišit v jiných datasetech
- také se opakuje atribut `sldb_rok`, který představuje rok sběru dat, ačkoli má tento atribut také pouze jednu hodnotu, může aspoň připomenout uživateli o jaký rok se jedná
- nejdůležitějším atributem je atribut `hodnota`, který obsahuje hodnotu daného záznamu (např. počet lidí s danou vlastností)

### Analýza datasetu "Obyvatelstvo podle pětiletých věkových skupin a pohlaví"

- celkový počet obyvatel: 10,524,167
    - celkový počet žen: 5,337,619 (50,72 % z celkového počtu)
    - celkový počet mužů: 5,168,548 (49,28 % z celkového počtu)
- celkový počet obyvatel podle věkových fází a pohlaví:
    - děti (0-9 let)
        - celkem: 1,110,656 (10,55 % z celkového počtu)
            - ženy: 541,695 (48,7 % z celkových dětí)
            - muži: 568,961 (51,3 % z celkových dětí)
    - dospívání (10-19 let)
        - celkem: 1,070,940 (10,18 % z celkového počtu)
            - ženy: 522,033 (48,7 % z celkového počtu dospívajících)
            - muži: 548,907 (51,3 % z celkového počtu dospívajících)
    - dospělí (20-59 let)
        - celkem: 5,577,915 (53 % z celkového počtu)
            - ženy: 2,713,576 (48,6 % z celkového počtu dospělých)
            - muži: 2,864,339 (51,4 % z celkového počtu dospělých)
    - staří (60 a více)
        - celkem: 2,764,656 (26,27 % z celkového počtu)
            - ženy: 1,560,315 (56,4 % z celkového počtu starších)
            - muži: 1,204,341 (43,6 % z celkového počtu starších)

- poměr počtu obyvatel v pozdním věku podle pohlaví a věku:
    - 50 - 59 let:
        - ženy: 668,842 (49,4 %)
        - muži: 685,659 (50,6 %)
    - 60 - 69 let:
        - ženy: 672,176 (52,3 %)
        - muži: 612,513 (47,7 %)
    - 70 - 79 let:
        - ženy: 595,034 (57,3 %)
        - muži: 442,963 (42,7 %)
    - 80 - 89 let:
        - ženy: 245,967 (64,9 %)
        - muži: 132,717 (35,1 %)
    - 90 - 99 let:
        - ženy: 46,602 (74,4 %)
        - muži: 16,037 (25,6 %)
    - 100 a více let
        - ženy: 536 (82,8 %)
        - muži: 111 (17,2 %)

- ačkoli je celkový počet žen větší než počet mužů, počet mužů do věku cca 60 je stále vyšší než počet žen, od pozdního věku (60+) je počet žen drasticky vyšší a lze spatřit, že se ženy dožívají vyššího věku, ve věku 90+ je žen téměř 3x víc než mužů
- výpočet průměrného věku je složitější, protože pracujeme s pětiletými skupinami a k tomu poslední skupina '100 let a více' může představovat více hodnot, proto bylo každé skupině přidělen její průměrný věk (např. u skupiny '0 - 4 let' je přiřazen průměrný věk 2) a skupině '100 let a více' byl pro férový výpočet přiřazen věk 102, výsledkem váženého průměru je věk 42,2 let
- medián věku neboli střední hodnota je věková skupina '40 - 44 let'
- zajímavé největší poměry pohlaví v obcích:
    - největší poměr mužů oproti ženám:
        - Lesní Jakubov
            - mužů: 743 (93 %)
            - žen: 54 (7 %)
            - poměr: **14:1** (zaokrouhleno)
    - největší poměr žen oproti mužům:
        - Zvíkovec
            - žen: 136 (66 %)
            - mužů: 68 (33 %) 
            - poměr: **2:1**

- nejmenší/největší
    - obec (bez městysů a měst)
        - nejméně zalidněna obec je Vysoká Lhota s 16 obyvateli a rozlohou 3,1 km2, počet obyvatel na km2 je 5
        - nejvíce zalidněna obec je Bystřice s 5,175 obyvateli a rozlohou 16 km2, počet obyvatel na km2 je 323,4
    - město
        - nejméně zalidněné město je Přebuz s 64 obyvateli a rozlohou 29,8 km2, poćet obyvatel na km2 je 2,1
        - nejvíce zalidněné město je Praha s 1,301,432 obyvateli a rozlohou 496,2 km2, poćet obyvatel na km2 je 2622,8
    - okres
        - nejméně zalidněný okres je Jeseník s 35471 obyvateli a rozlohou 719 km2, počet obyvatel na km2 je 49,3
        - nejvíce zalidněný okres je Praha s 1,301,432 obyvateli a rozlohou 496,2 km2, počet obyvatel na km2 je 2622,8
    - celkově
        - nejmenší hustota zalidnění (obyv/km2) je v Modravě - 1,09 obyv na km2
        - největší hustota zalidnění (obyv/km2) je v Praze - 2622,9 obyv na km2

- největší/nejmenší poměr dětí a starších v obcích
    - děti
        - největší poměr dětí oproti celkovému počtu obyvatel
            - Horní Myslová - 25 %
        - nejmenší poměr dětí oproti celkovému počtu obyvatel
            - více (např. Čejkovice) - 0 %
    - staří
        - největší poměr starých oproti celkovému počtu obyvatel
            - Vysoká Lhota - 81,25 %
        - nejmenší poměr starých oproti celkovému počtu obyvatel
            - Lesní Jakubov - 6,1 %

### Analýza datasetu "Obyvatelstvo podle věku, pohlaví a vzdělání"

- hned první výtka - dataset má 125 MB ve formátu CSV, obtížné načítání, zvlášť na starých PC, i na moderním PC (6 fyz. jader, 16 gb ram) to trvá skoro 4 minuty převést z CSV do XLSX

- oproti minulému datasetu, který by měl mít podobné atributy, má navíc text územního typu (zda je to obec, město apod.), to velice napomáhá, minule bylo potřeba zjiśťovat kódy území a podle toho zjistit, zd ase jedná např. o obec nebo město
- obsahuje "zbytečné" atributy, které mají vždy jen jednu hodnotu a opakuje se na každém řádku, to zabírá neskutečně moc místa a pomohlo by zredukovat obtížnost načítání dat a velikost souboru
    - pokud odstraníme atributy, které mají vždy jednu stejnou hodnotu, tak má toto "zredukované" CSV o 55 % menší velikost (131 MB -> 59 MB)
    - při převodu zpátky do CSV přes Excel (kvůli smazání sloupců) došlo ke změně formátování z UTF-8 na ANSI, Excel neumí nastavit formátování CSV při ukládání, byl použit Notepad++ pro převod zpět do UTF-8
- jiné věkové skupiny oproti minulému datasetu, v tomto datasetu nejsou zahrnuty děti, tentokrát jsou zde pouze dvě věkové skupiny a to '15 - 64 let' a '65 a více let', věková skupina pod 15 let zde není obsažena, dává smysl pro vzdělání
- nově se zde vyskytují hodnoty typu 'neuvedeno', což znamená, že osoba nechtěla odpovědět, jedná se o atribut vzdělání, dává smysl
- u věku se nevyskytuje "celková" hodnota, což může ztížit počítaní celkových hodnot, v minulém datasetu byly obsaženy celkové hodnoty
- celkový počet záznamů o vzdělání (i těch, kteří neuvedli) neodpovídá celkovému počtu obyvatel ČR, k dispozici je pouze cca 8,8 miliónů záznamů - zřejmě kvůli dětem 

Hlavní informace a postřehy:

- dataset má cca **564 tisíc** záznamů
- dohromady zjištěných obyvatel je **8,832,407**, zbytek obyvatel jsou pravděpodobně děti, které do tohoto datasetu nezapadají
- z celkových **8,832,407** obyvatel nechtělo odpovědět na typ vzdělání **511,378**, což je **5,7 %**
- typy vzdělání a jejich poměr na celkovém počtu v ČR (výběr nejzákladnějších):
    - bez vzdělání (**0,64 %** z celkového počtu)
        - ženy: 27,440 (48,9 %)
        - muži: 28,660 (51,1 %)
    - neúplné základní vzdělání (**1,3 %** z celkového počtu)
        - ženy: 59,781 (53,2 %)
        - muži: 52,684 (46,8 %)
    - základní vzdělání (**11,3 %** z celkového počtu)
        - ženy: 604,955 (60,8 %)
        - muži: 390,440 (39,2 %)
    - úplné střední všeobecné vzdělání (gymnázia) (**12.7 %** z celkového počtu)
        - ženy: 676,678 (60.3 %)
        - muži: 444,829 (39.7 %)
    - nižší střední a střední vzdělání (**31 %** z celkového počtu)
        - ženy: 1,182,747 (43,2 %)
        - muži: 1,554,236 (56,8 %)
    - úplné střední odborné vzdělání (**15,6 %** z celkového počtu)
        - ženy: 698,167 (50,8 %)
        - muži: 676,965 (49,2 %)
    - vysokoškolské bakalářské vzdělání (**3,8 %** z celkového počtu)
        - ženy: 200,884 (60,7 %)
        - muži: 130,105 (39,3 %)
    - vysokoškolské magisterské vzdělání (**13 %** z celkového počtu)
        - ženy: 593,633 (51,9 %)
        - muži: 550,699 (48,1 %)
    - vysokoškolské doktorské  vzdělání (**0,9 %** z celkového počtu)
        - ženy: 28,679 (37,2 %)
        - muži: 48,407 (62,8 %)
- nejnižší a největší poměr vysokoškolsky vzdělaných lidí v krajích, okresech a teoretických městech (obce nad 3000 lidí):
    - pozn. vysokoškolským vzděláním se zde myslí dokončené bakalářské, magisterské a doktorské studium na VŠ
    - pozn. obce nad 3000 lidí byly vydedukovány z předchozího datasetu, kde se nachází celkový počet lidí v dané obci, čísla pod 3000 znamenají počet obyvatel bez dětí
    - nejnižší poměr:
        - obec Jiříkov - z celkové počtu 2,918 je 130 (**3,75 %**) vysokoškolsky vzdělaných
        - okres Sokolov - z celkové počtu 70,896 je 5,291 (**7,46 %**) vysokoškolsky vzdělaných
        - kraj Karlovarský - z celkové počtu 236,517 je 22,803 (**9,64 %**) vysokoškolsky vzdělaných
    - nejvyšší poměr:
        - obec Černošice - z celkové počtu 6,172 je 2,303 (**29,9 %**) vysokoškolsky vzdělaných
        - okres Praha - z celkové počtu 1,102,063 je 371,351 (**33,7 %**) vysokoškolsky vzdělaných
        - kraj Hlavní město Praha - z celkové počtu 1,102,063 je 371,351 (**33,7 %**) vysokoškolsky vzdělaných
- nejnižší a největší poměr nevzdělaných lidí v krajích, okresech a teoretických městech (obce nad 3000 lidí):
    - pozn. obce nad 3000 lidí byly vydedukovány z předchozího datasetu, kde se nachází celkový počet lidí v dané obci, čísla pod 3000 znamenají počet obyvatel bez dětí
    - nejnižší poměr:
        - obec Jirny - z celkové počtu 2,517 je 0 (**0 %**) bez vzdělání
        - okres Praha - z celkové počtu 1,102,063 je 3,916 (**0,35 %**) bez vzdělání
        - kraj Hlavní město Praha - z celkové počtu 1,102,063 je 3,916 (**0,35 %**) bez vzdělání
    - nejvyšší poměr:
        - obec Slatiňany - z celkové počtu 3,359 je 108 (**2,7 %**) bez vzdělání
        - okres Most - z celkové počtu 88,475 je 1,343 (**1,5 %**) bez vzdělání
        - kraj Ústecký - z celkové počtu 662,139 je 7,794 (**1,17 %**) bez vzdělání

### Analýza datasetu "Obyvatelstvo podle náboženství"

- dataset má kolem 75 MB, obtížné načítání

- dataset opět obsahuje "zbytečné atributy", jedná se o atributy, které se opakují pro každý záznam (mají jednu a tu stejnou hodnotu v každém záznamu) - jedná se o rok, datum a další, zmínit, že sice se může zdát, že nemají použití, tak mohou pomoci při spojení s jinými datasety

- obsahuje hodnoty jako neuvedeno (dobrovolná položka ve formuláři), ale obsahuje i uvedeno, ale nespecikováno, také obsahuje "jiné"

- obsahuje i celkové počty pro dané území

Hlavní informace a postřehy:

- 531 tisíc záznamů
- dataset rozlišuje 74 různých typů vír/církví, obsahuje ale i hodnotu "jiné", takže vír se v Česku vyskytuje pravděpodobně více
- nejvýznamnějším náboženstvím je pravděpodobně Církev římskokatolická s 741,019 věřícími
- náboženství neuvedlo 3,162,540 lidí (30 %)
- nevěřících je 5,027,141 (47,7 %)
- věřících, kteří se nehlásí k žádné církvi nebo nábož. společnosti, je 960,201
- věřících církvi bez specifikované církve je 65,567
- pouze jeden jediný člověk se hlásí k Společenství buddhismu v České republice (nejedná se o obecný buddhismus)
- dataset obsahuje záznamy o víře v rytíře Jedi a Sith (fiktivní postavy ze série Star Wars), v tyto víry věří dohromady 21,539 lidí

### Analýza datasetu "Obyvatelstvo podle občanství"